# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-22 05:42:07] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-22 05:42:07] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-22 05:42:07] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-22 05:42:07] INFO lfm2.py:102: Registered SGLang Lfm2Config to override HuggingFace's version


[2026-01-22 05:42:10] INFO server_args.py:1724: Attention backend not specified. Use fa3 backend by default.


...

Capturing batches (bs=128 avail_mem=74.80 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=80 avail_mem=74.67 GB):  20%|██        | 4/20 [00:00<00:01,  9.11it/s]

Capturing batches (bs=32 avail_mem=74.64 GB):  50%|█████     | 10/20 [00:00<00:00, 18.28it/s]

Capturing batches (bs=4 avail_mem=74.61 GB):  80%|████████  | 16/20 [00:01<00:00, 20.68it/s]

Capturing batches (bs=1 avail_mem=74.60 GB): 100%|██████████| 20/20 [00:01<00:00, 17.88it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Alexander, a software engineer, and I am a Java programmer. I have been working with Java for a long time and have gained a lot of experience through it. I would like to learn more about the Java programming language and its applications. Can you provide me with some insights on the Java programming language and its current applications? Please include Java 11, 12, and 13, and their functionalities and applications.
Sure, I can provide you with some insights on the Java programming language and its current applications. Here is a brief overview of the different Java versions and their functionalities:

1. Java 11:
-
Prompt: The president of the United States is
Generated text:  now trying to choose a new birthday cake for the nation. He wants to have a cake with a special ingredient that changes color based on the year. The cake is to have 12 candles, and the color of the candles will change every year based on the year. If the year is divisib

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [job title] at [company name], and I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [job title] at [company name], and I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [job title] at [company name], and I'm excited to meet you and learn more about you. What

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. It is also famous for its rich history, including the French Revolution and the French Revolution Museum. Paris is a bustling city with a diverse population and a rich cultural heritage. It is the largest city in France and the third-largest city in the world by population. It is also a major financial center and a major tourist destination. Paris is known for its cuisine, art, and fashion, and is home to many world-renowned museums, theaters, and landmarks. It is a city that is constantly evolving and changing,

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction and impact on society. Here are some of the most likely trends that are expected to shape the future of AI:

1. Increased use of AI in healthcare: AI is already being used in healthcare to improve patient outcomes and reduce costs. As AI technology continues to improve, we can expect to see even more widespread use of AI in healthcare, with the potential to revolutionize the way we diagnose and treat diseases.

2. Increased use of AI in finance: AI is already being used in finance to improve fraud detection, risk management, and investment decision-making



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I am an [occupation]. I enjoy [way you can describe your current hobby/interest/interest in life that you enjoy doing]. I have always been fascinated by [reason why you are so passionate about your chosen career]. I am eager to learn more about [occupation], to gain a deeper understanding of [reason why you are passionate about this occupation]. I am confident that my background and experience in [occupation] will help me achieve my goals, no matter what they may be.

You have mentioned that you enjoy [way you can describe your current hobby/interest/interest in life that you enjoy doing]. Can you

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the iconic city renowned for its iconic landmarks such as the Eiffel Tower and Notre-Dame Cathedral. 

This text is concise and factual, providing a 

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

].

 I

...

 also

 more

 adaptable

 and

 effective

In [6]:
llm.shutdown()